In [201]:
import pandas as pd
import numpy as np
import random
import difflib


df = pd.read_csv('./data/training.txt', sep="\t", header = None)
df.columns  = [1, 2, 3, 4, 5, 6, 7, "class"]

df = df.applymap(lambda x: True if x == 2 else False)

        

In [233]:
def decisionTreeLearning(examples, attributes, parent_examples):
    attributes = attributes
    #print examples
    if len(examples) == 0:
        #print "parent_exmaples: ", parent_examples
        #print "no more examples, plularity value parents: ", pluralityValue(parent_examples)
        return pluralityValue(parent_examples)
    elif allSameClass(examples):
        #print "all same class: ", getClass(examples)
        return getClass(examples)
    elif len(attributes) == 0:
        #print "no more attributes, pluarityvaklue: ", pluralityValue(examples)
        return pluralityValue(examples)
    else:
        parent_examples = examples
        next_att = importance(examples, attributes)
        #print "else, next att: ", next_att
        next_attributes = attributes
        next_attributes.remove(next_att)
        #print "reamining attributes: ", attributes
        tree = [next_att, {True: decisionTreeLearning(examples[examples[next_att] == True], next_attributes, examples), False: decisionTreeLearning(examples[examples[next_att] == False], next_attributes, parent_examples)} ]
        return tree

def dumbDecisionTreeLearning(examples, attributes, parent_examples):
    if len(examples) == 0:
        return pluralityValue(parent_examples)
    elif allSameClass(examples):
        return getClass(examples)
    elif len(attributes) == 0:
        return pluralityValue(examples)
    else:
        parent_examples = examples
        next_att = random.sample(set(attributes), 1)[0]
        next_attributes = attributes
        next_attributes.remove(next_att)
        tree = [next_att, {True: dumbDecisionTreeLearning(examples[examples[next_att] == True], next_attributes, examples), False: dumbDecisionTreeLearning(examples[examples[next_att] == False], next_attributes, parent_examples)} ]
        return tree

def pluralityValue(examples):
    true_count = (examples["class"]).sum()
    false_count = len(examples) - true_count
    if true_count > false_count:
        return True
    elif true_count < false_count:
        return False
    else:
        return random.sample(set([True, False]), 1)[0]

        
def allSameClass(examples):
    return len(examples["class"].value_counts()) == 1
    
def getClass(examples):
    return examples["class"].iloc[0]


def gain(examples, attribute):
    pos = getPosCount(examples)
    return B(pos/float(len(examples))) - remainder(examples, attribute)
    
    
def B(q):
    if q <= 0 or q == 1 :
        return 0
    result = - (q*np.log2(q) + (1 - q)*np.log2(1 - q))
    return result

def remainder(examples, attribute):
    
    class_pos_count = getPosCount(examples)
        
    att_pos_split = examples[examples[attribute] == True]
    att_neg_split =  examples[examples[attribute] == False]
    
    att_pos_class_pos_count = getPosCount(att_pos_split)
    att_neg_class_pos_count = getPosCount(att_neg_split)
        
    remainder_att_pos = 0 if len(att_pos_split) == 0 else (len(att_pos_split)/float(len(examples))) * B(att_pos_class_pos_count/float(len(att_pos_split))) 
    remainder_att_neg = 0 if len(att_neg_split) == 0 else (len(att_neg_split)/float(len(examples))) * B(att_neg_class_pos_count/float(len(att_neg_split)))
    return remainder_att_pos + remainder_att_neg  
    
def getPosCount(examples):
    return (examples["class"]).sum()

def importance(examples, attributes):
    gains_dicc = {att: gain(examples, att) for att in attributes}
    #print gains_dicc
    max_gain = max(gains_dicc.values())
    #print max_gain
    important_atts = filter(lambda att: gains_dicc[att] >= max_gain, attributes)
    return random.sample(set(important_atts), 1)[0]


#dumb_tree = dumbDecisionTreeLearning(df, attributes, df)

False


,1,2,3,4,5,6,7,class
0,False,False,True,True,False,False,False,False
1,False,True,True,False,False,False,False,False
2,False,True,False,True,True,True,True,False
3,False,False,True,True,True,False,False,False
4,True,True,False,False,True,True,False,True
5,False,True,True,False,True,False,True,False
6,False,False,False,False,False,False,True,False
7,False,True,False,False,False,True,False,False
8,True,False,True,False,False,False,True,True
9,False,True,True,True,True,False,False,False


In [210]:

def classify(item, tree):
    while True:
        att = tree[0]
        value = item[att]
        dicc = tree[1]
        tree = dicc[value]
        if not isinstance(tree, list):
            return tree
            



1        False
2        False
3         True
4         True
5        False
6        False
7        False
class    False
Name: 0, dtype: bool


In [212]:
test_df = pd.read_csv('./data/test.txt', sep="\t", header = None)
test_df.columns = [1, 2, 3, 4, 5, 6, 7, "class"]

test_df = test_df.applymap(lambda x: True if x == 2 else False)

def getResults(dumb_tree, test_df):
    results = []
    for index, row in test_df.iterrows():
        results.append(classify(row, dumb_tree))
    return results

def assessPerformance(results, actual_results):
    sm=difflib.SequenceMatcher(None,results,actual_results)
    return sm.ratio()


dumbTreePerformances = []
for i in range(100):
    dumb_tree = dumbDecisionTreeLearning(df, [1, 2, 3, 4, 5, 6, 7 ], df)
    results = getResults(dumb_tree, test_df)
    actual_results = test_df["class"].tolist()
    dumbTreePerformances.append(assessPerformance(results, actual_results))

print (sum(dumbTreePerformances)/len(dumbTreePerformances))


test_df

0.6375
1        False
2        False
3         True
4         True
5        False
6        False
7        False
class    False
Name: 0, dtype: bool


,1,2,3,4,5,6,7,class
0,False,True,False,False,True,True,False,False
1,False,False,True,False,True,True,False,False
2,False,False,True,True,True,False,True,False
3,False,True,True,True,False,True,False,False
4,False,False,False,False,False,True,True,False
5,True,False,True,False,False,False,False,True
6,True,True,True,True,True,True,True,False
7,True,False,True,False,True,True,False,True
8,True,False,True,True,False,True,False,True
9,True,True,False,True,False,True,False,True


In [225]:
item = test_df.loc[9]
print item
print classify(item, tree)

1         True
2         True
3        False
4         True
5        False
6         True
7        False
class     True
Name: 9, dtype: bool
False


In [208]:
#testing tree with training data
tree = decisionTreeLearning(df, [1, 2, 3, 4, 5, 6, 7], df)
results = getResults(tree, df)
actual_results = df["class"].tolist()

actual_test_results = df["class"].tolist()
print assessPerformance(results, actual_test_results)
tree




gholaaa
        1      2      3      4      5      6      7  class
0   False  False   True   True  False  False  False  False
1   False   True   True  False  False  False  False  False
2   False   True  False   True   True   True   True  False
3   False  False   True   True   True  False  False  False
4    True   True  False  False   True   True  False   True
5   False   True   True  False   True  False   True  False
6   False  False  False  False  False  False   True  False
7   False   True  False  False  False   True  False  False
8    True  False   True  False  False  False   True   True
9   False   True   True   True   True  False  False  False
10   True  False  False  False   True   True  False  False
11  False  False   True   True  False   True  False  False
12  False   True   True   True  False  False  False  False
13   True   True   True  False   True  False   True  False
14  False   True   True  False  False   True   True  False
15   True  False   True  False  False   True  Fa

[1,
 {False: False,
  True: [5,
   {False: False,
    True: [6,
     {False: False,
      True: [7,
       {False: True,
        True: [4,
         {False: [2, {False: [3, {False: False, True: True}], True: True}],
          True: True}]}]}]}]}]

In [230]:
item = test_df.loc[10]
print item
print classify(item, tree)

1         True
2         True
3        False
4         True
5        False
6         True
7         True
class     True
Name: 10, dtype: bool
False


In [101]:
#testing tree with test data
tree = decisionTreeLearning(df, [1, 2, 3, 4, 5, 6, 7], df)
results = getResults(tree, test_df)
actual_results = test_df["class"].tolist()
print assessPerformance(results, actual_results)
tree

else, next att:  1
attributes:  [1, 2, 3, 4, 5, 6, 7]
else, next att:  5
attributes:  [2, 3, 4, 5, 6, 7]
else, next att:  6
attributes:  [2, 3, 4, 6, 7]
else, next att:  7
attributes:  [2, 3, 4, 7]
else, next att:  3
attributes:  [2, 3, 4]
all same class:  True
else, next att:  2
attributes:  [2, 4]
all same class:  True
all same class:  False
else, next att:  4
attributes:  [4]
no more attributes, pluarity parents:  True
no more attributes, pluarity parents:  False
no more attributes, pluarity parents:  True
no more attributes, pluarity parents:  False
all same class:  False
0.571428571429


[1,
 {False: False,
  True: [5,
   {False: False,
    True: [6,
     {False: False,
      True: [7,
       {False: [4, {False: False, True: True}],
        True: [3, {False: [2, {False: False, True: True}], True: True}]}]}]}]}]

In [234]:

treePerformances = []
for i in range(100):
    tree = decisionTreeLearning(df, [1, 2, 3, 4, 5, 6, 7], df)
    results = getResults(tree, test_df)
    actual_results = test_df["class"].tolist()
    treePerformances.append(assessPerformance(results, actual_results))

treePerformances
print (sum(treePerformances)/len(treePerformances))


0.690357142857


In [174]:
len(test_df)

28